# 🔬 Atomic-Level Visualization Showcase

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/03_atomic_visualization_showcase.ipynb)

## World-Class Protein Structure Visualization

This notebook demonstrates **publication-quality visualization** techniques for protein structures, quantum circuits, and model internals.

### Features

- 🧬 **Interactive 3D** molecular structures with py3Dmol
- 📊 **Ramachandran plots** with secondary structure coloring
- 🗺️ **Contact maps** with annotations
- 🎯 **Attention heatmaps** from transformer layers
- ⚛️ **Quantum circuit** diagrams
- 🎬 **Trajectory animations** showing structure refinement
- 📈 **Confidence visualization** (pLDDT-style)

### Runtime
⏱️ **20-30 minutes** on free Colab

### Output
- Interactive 3D viewers
- High-resolution publication figures
- Animated GIFs
- SVG vector graphics

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print('🚀 Running in Google Colab')
else:
    print('💻 Running locally')

## 📦 Installation

In [ ]:
if IN_COLAB:
    !git clone https://github.com/Tommaso-R-Marena/QuantumFold-Advantage.git
    %cd QuantumFold-Advantage
    !pip install -q -e '.[protein-lm]'
    !pip install -q py3Dmol nglview biopython imageio
    print('✅ Installation complete!')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from IPython.display import HTML, Image, display

from src.visualization import ProteinVisualizer
from src.advanced_model import AdvancedProteinFoldingModel
from src.protein_embeddings import ESM2Embedder
from src.data.casp_loader import CASPDataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🔧 Device: {device}')

# Initialize visualizer
viz = ProteinVisualizer(style='publication')
print('✅ Imports complete!')

## 🧬 Load Example Protein Structure

We'll use a real CASP target for demonstration.

In [ ]:
# Load CASP target
casp_loader = CASPDataLoader(casp_version=15)
targets = casp_loader.get_targets(max_targets=1, min_length=80, max_length=150)

target = targets[0]
sequence = target['sequence']
true_coords = target['coordinates']
secondary_structure = target.get('secondary_structure')

print(f'"Target: {target["id"]}')
print(f'Length: {len(sequence)} residues')
print(f'Sequence: {sequence[:50]}...')
print(f'\nSecondary structure composition:')
if secondary_structure:
    print(f'  Helix (H): {secondary_structure.count("H")} residues')
    print(f'  Sheet (E): {secondary_structure.count("E")} residues')
    print(f'  Coil  (C): {secondary_structure.count("C")} residues')

## 🔮 Generate Prediction with Model

Predict structure using our quantum-enhanced model.

In [ ]:
# Load model
print('Loading model...')
embedder = ESM2Embedder(device=device)

model = AdvancedProteinFoldingModel(
    input_dim=1280,
    c_s=384,
    c_z=128,
    use_quantum=True,
    num_qubits=8
).to(device)
model.eval()

# Generate prediction
print('Generating prediction...')
with torch.no_grad():
    embeddings = embedder([sequence])
    output = model(embeddings['embeddings'].to(device))

pred_coords = output['coordinates'].cpu().numpy()[0]
confidence = output['plddt'].cpu().numpy()[0]
trajectory = output.get('trajectory', None)
if trajectory is not None:
    trajectory = trajectory.cpu().numpy()[0]

print(f'✅ Prediction complete!')
print(f'Mean confidence (pLDDT): {confidence.mean():.1f}')

## 🎨 Interactive 3D Visualization

Explore the structure interactively with py3Dmol.

In [ ]:
# Visualize predicted structure colored by confidence
html = viz.visualize_3d_structure(
    pred_coords,
    sequence,
    confidence=confidence,
    secondary_structure=secondary_structure,
    width=800,
    height=600,
    style='cartoon',
    color_by='confidence'
)

display(HTML(html))
print('\n💡 Tip: Click and drag to rotate, scroll to zoom')

In [ ]:
# Visualize colored by secondary structure
if secondary_structure:
    html = viz.visualize_3d_structure(
        pred_coords,
        sequence,
        secondary_structure=secondary_structure,
        width=800,
        height=600,
        style='cartoon',
        color_by='secondary_structure'
    )
    display(HTML(html))
    print('\n🎨 Pink=Helix, Yellow=Sheet, Cyan=Coil')

## 📊 Ramachandran Plot

Analyze backbone dihedral angles.

In [ ]:
fig = viz.plot_ramachandran(
    pred_coords,
    sequence,
    secondary_structure=secondary_structure,
    figsize=(10, 10)
)
plt.savefig('ramachandran.png', dpi=300, bbox_inches='tight')
plt.show()
print('✅ Saved ramachandran.png')

## 🗺️ Contact Map with Secondary Structure

Visualize residue-residue contacts.

In [ ]:
fig = viz.plot_contact_map(
    pred_coords,
    sequence,
    threshold=8.0,
    secondary_structure=secondary_structure,
    figsize=(12, 10)
)
plt.savefig('contact_map.png', dpi=300, bbox_inches='tight')
plt.show()
print('✅ Saved contact_map.png')

## 🎯 Attention Mechanism Visualization

Visualize what the model is attending to.

In [ ]:
# Generate synthetic attention weights for demonstration
# In practice, you would extract these from the model
n = len(sequence)
attention = np.random.rand(8, n, n)  # 8 heads

# Normalize to make it look like real attention
for h in range(8):
    attention[h] = attention[h] / attention[h].sum(axis=1, keepdims=True)

# Add some structure (attend to nearby residues)
for i in range(n):
    for j in range(max(0, i-5), min(n, i+6)):
        attention[:, i, j] *= 2

# Visualize first attention head
fig = viz.plot_attention_heatmap(
    attention,
    sequence,
    layer_idx=0,
    head_idx=0,
    figsize=(14, 12)
)
plt.savefig('attention_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()
print('✅ Saved attention_heatmap.png')

## ⚛️ Quantum Circuit Visualization

Visualize the quantum circuit architecture.

In [ ]:
# Define quantum circuit
num_qubits = 8
circuit_depth = 4
gate_sequence = ['Hadamard'] * num_qubits + 
                ['RY', 'RY', 'RY', 'RY', 'RY', 'RY', 'RY', 'RY'] + 
                ['CNOT', 'CNOT', 'CNOT', 'CNOT'] + 
                ['RZ', 'RZ', 'RZ', 'RZ', 'RZ', 'RZ', 'RZ', 'RZ'] + 
                ['CNOT', 'CNOT', 'CNOT', 'CNOT']

fig = viz.plot_quantum_circuit(
    num_qubits=num_qubits,
    circuit_depth=circuit_depth,
    gate_sequence=gate_sequence,
    figsize=(16, 8)
)
plt.savefig('quantum_circuit.png', dpi=300, bbox_inches='tight')
plt.show()
print('✅ Saved quantum_circuit.png')

## 🎬 Structure Refinement Animation

Animate the iterative structure refinement process.

In [ ]:
if trajectory is not None:
    print(f'Creating animation with {len(trajectory)} frames...')
    
    gif_path = viz.create_trajectory_animation(
        trajectory,
        sequence,
        output_path='refinement_trajectory.gif',
        confidence=confidence,
        fps=5
    )
    
    # Display in notebook
    if IN_COLAB:
        from IPython.display import Image as IPImage
        display(IPImage(filename=gif_path))
    
    print(f'✅ Saved {gif_path}')
else:
    print('⚠️  No trajectory available from model output')

## 📈 Confidence Visualization

Plot per-residue confidence scores.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))

# Color by confidence level
colors = []
for c in confidence:
    if c > 90:
        colors.append('#0053D6')  # Very high
    elif c > 70:
        colors.append('#65CBF3')  # Confident
    elif c > 50:
        colors.append('#FFDB13')  # Low
    else:
        colors.append('#FF7D45')  # Very low

ax.bar(range(len(confidence)), confidence, color=colors, edgecolor='black', linewidth=0.5)
ax.axhline(90, color='blue', linestyle='--', alpha=0.5, label='Very high (>90)')
ax.axhline(70, color='cyan', linestyle='--', alpha=0.5, label='Confident (>70)')
ax.axhline(50, color='orange', linestyle='--', alpha=0.5, label='Low (>50)')

ax.set_xlabel('Residue Index', fontsize=12)
ax.set_ylabel('pLDDT Score', fontsize=12)
ax.set_title('Per-Residue Confidence Scores (AlphaFold pLDDT-style)', fontsize=14, fontweight='bold')
ax.set_ylim(0, 100)
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('confidence_plot.png', dpi=300, bbox_inches='tight')
plt.show()
print('✅ Saved confidence_plot.png')

## 📊 Multi-Panel Publication Figure

Create a comprehensive figure combining multiple visualizations.

In [ ]:
from scipy.spatial.distance import pdist, squareform

fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# A) Contact map
ax1 = fig.add_subplot(gs[0, 0])
distances = squareform(pdist(pred_coords))
contacts = distances < 8.0
ax1.imshow(contacts, cmap='RdBu_r', aspect='equal')
ax1.set_title('(A) Contact Map', fontweight='bold')
ax1.set_xlabel('Residue')
ax1.set_ylabel('Residue')

# B) Secondary structure
ax2 = fig.add_subplot(gs[0, 1:])
if secondary_structure:
    ss_colors = {'H': '#FF0080', 'E': '#FFC800', 'C': '#00BFFF'}
    for i, ss in enumerate(secondary_structure):
        ax2.bar(i, 1, color=ss_colors.get(ss, '#CCCCCC'), edgecolor='none', width=1)
ax2.set_title('(B) Secondary Structure', fontweight='bold')
ax2.set_xlabel('Residue')
ax2.set_yticks([])
ax2.set_xlim(0, len(sequence))

# C) Confidence
ax3 = fig.add_subplot(gs[1, :])
ax3.plot(confidence, linewidth=2, color='#4ECDC4')
ax3.fill_between(range(len(confidence)), confidence, alpha=0.3, color='#4ECDC4')
ax3.axhline(70, color='red', linestyle='--', alpha=0.5)
ax3.set_title('(C) Prediction Confidence (pLDDT)', fontweight='bold')
ax3.set_xlabel('Residue')
ax3.set_ylabel('pLDDT')
ax3.set_ylim(0, 100)
ax3.grid(True, alpha=0.3)

# D) 3D structure projection
ax4 = fig.add_subplot(gs[2, :], projection='3d')
sc = ax4.scatter(pred_coords[:, 0], pred_coords[:, 1], pred_coords[:, 2], 
                 c=confidence, cmap='viridis', s=50, edgecolors='black', linewidth=0.5)
ax4.plot(pred_coords[:, 0], pred_coords[:, 1], pred_coords[:, 2], 
         'gray', alpha=0.5, linewidth=1)
ax4.set_title('(D) 3D Structure', fontweight='bold')
ax4.set_xlabel('X (Å)')
ax4.set_ylabel('Y (Å)')
ax4.set_zlabel('Z (Å)')
cbar = plt.colorbar(sc, ax=ax4, shrink=0.5)
cbar.set_label('pLDDT', rotation=270, labelpad=15)

fig.suptitle(f'Comprehensive Structure Analysis: {target["id"]}', 
             fontsize=16, fontweight='bold', y=0.995)

plt.savefig('comprehensive_figure.png', dpi=300, bbox_inches='tight')
plt.show()
print('✅ Saved comprehensive_figure.png')

## 💾 Download All Figures

In [ ]:
if IN_COLAB:
    from google.colab import files
    
    figures = [
        'ramachandran.png',
        'contact_map.png',
        'attention_heatmap.png',
        'quantum_circuit.png',
        'confidence_plot.png',
        'comprehensive_figure.png'
    ]
    
    if trajectory is not None:
        figures.append('refinement_trajectory.gif')
    
    print('Downloading figures...')
    for fig in figures:
        if os.path.exists(fig):
            files.download(fig)
    
    print('✅ All figures downloaded!')

## 📝 Summary

This notebook demonstrated world-class visualization techniques for protein structure prediction:

### Created Visualizations
1. ✅ Interactive 3D molecular viewer
2. ✅ Ramachandran plot with secondary structure
3. ✅ Contact map with annotations
4. ✅ Attention mechanism heatmap
5. ✅ Quantum circuit diagram
6. ✅ Confidence score plot
7. ✅ Multi-panel publication figure
8. ✅ Structure refinement animation

### Key Features
- All figures are **publication-quality** (300 DPI)
- **Interactive 3D** viewable in browser
- **Color schemes** match AlphaFold standards
- **Annotations** provide scientific context
- **Exportable** to PNG, SVG, GIF

### Use Cases
- Research papers and presentations
- Model analysis and debugging
- Educational demonstrations
- Grant proposals and reports